# Introduction to LiberTEM

Inspired by https://github.com/LiberTEM/LiberTEM/blob/master/examples/Introduction%20to%20UDFs.ipynb and https://libertem.github.io/LiberTEM-blobfinder/strainmap-SiGe.html.

This notebook should be self-contained, but requires an internet connection to download a public Merlin Medipix dataset. The dataset is extracted into the current directory under the folder `'./dataset'`.

## What is LiberTEM ???

LiberTEM is an open-source library for high-speed data processing which is specialised towards 4D-STEM datasets, but in fact is capable of efficiently carrying out any task where the computation can be performed independently on different pieces of the dataset before final merging into a result.

The main project page can be found here https://libertem.github.io/LiberTEM/.

The project is primarily run by the FZ Jülich research centre in Germany, but is open-source so is freely available and anyone can contribute or request features.

## Documentation

The documentation is mainly found on the above site, though for completeness also check out the GitHub page where the code is stored and all discussion about features / bugs etc happens: https://github.com/LiberTEM/LiberTEM/.

## Concepts

#### Partitions

LiberTEM is based on a `map--reduce` programming model, which means the data to process is broken up into chunks (partitions) which can be independently processed (*map*) into intermediate results, which are then combined into a single result (usually) much smaller than the datset itself (*reduce*).

#### Clusters

LiberTEM is based on the Dask framework for parallel computing (https://dask.org/), which makes it very flexible and easy to scale from a single machine to multi-machine / cluster settings. In addition, LiberTEM supports running functions using CuPy (https://cupy.dev/) to access GPU-computing resources without changes to the code, allowing execution on hybrid CPU / GPU clusters with minimal user input. A cluster is the general term for the computing backend of LiberTEM (even if it is confined to one machine), and is composed of one-or-more workers which each process some subset of dataset partitions as required.

#### User Defined Functions (UDFs)

UDFs are LiberTEM's mechanism to specify a function to run on a dataset. The format, while a bit more verbose than a simple Python function, enables the automatic scaling and partitioned processing described above. UDFs also allow the system to achieve maximum possible performance, particularly with respect to accessing data on the disk or network, which is often the slowest part of a computation when datasets are very large.

#### Lazy computation

LiberTEM relies heavily on *lazy computation* which is the concept that the system does the minimum necessary at any moment in order to reduce resource consumption. Reading data and computation is done at the last possible moment and resources are liberated as quickly as possible, meaning that no time or resource is wasted.

More concretely, when you load a dataset with LiberTEM, nothing actually happens except building the *recipe* to **later** load the data when it is needed. The hard drive or network is barely solicitied. During computation, only the data which is currently being processed is loaded from disk; once computation on a chunk of data is complete it is removed from memory. As a result the RAM usage of the program rarely goes past a few gigabytes even when processing multi-hundred GB datasets.

### This notebook:

This notebook demonstrates:

- Creating a LiberTEM compute context / cluster
- Loading a dataset
- Running a simple function over a dataset
- Running a full User Defined Function (UDF)
- Running Brightfield, Annular Dark Field and CoM analyses
- Diffraction spot matching
- Creating and running a GPU-enabled custom UDF performing the Fourier Transform of each frame

Firstly some setup:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def _plot_img(array, figsize=(16, 16), title=None, ax=None, fig=None, cbar=False, **kwargs):
    if ax is None:
        fig, ax = plt.subplots(figsize=figsize)
    im = ax.imshow(array, cmap=kwargs.pop('cmap', 'gray'), **kwargs)
    if title is not None:
        ax.set_title(title)
    if cbar:
        assert fig is not None
        location = cbar if isinstance(cbar, str) else 'right'
        fig.colorbar(im, ax=ax, location=location, fraction=0.05)
    return ax

Download and extract a public dataset from Zenodo (https://zenodo.org/record/5113449), this may take a while:

In [ ]:
import pathlib
dataset_path = pathlib.Path('./dataset/20200518 165148/default.hdr')

In [ ]:
import urllib.request
import zipfile

dataset_path.parent.parent.mkdir(exist_ok=True)
if not dataset_path.is_file():
    zip_path = dataset_path.parent.parent / '20200518 165148.zip'
    URL = "https://zenodo.org/record/5113449/files/20200518%20165148.zip"
    print('Downloading...')
    urllib.request.urlretrieve(URL, filename=zip_path)
    print('Done')
    print('Extracting')
    if dataset_path.parent.is_dir():
        _ = [f.unlink() for f in dataset_path.parent.iterdir()]
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dataset_path.parent.parent)
    print('Done')
    zip_path.unlink()
else:
    print('Datset already present')

### Importing LiberTEM

Importing libertem is similar to Hyperspy (and many other Python packages). The `lt` name is customary and is used to create most of the necessary objects. Here I also load `cluster_spec` and `DaskJobExecutor` because I want to demonstrate customising the size of the compute resources.

In [ ]:
import libertem.api as lt

### Specify the cluster size (optional but useful)

We can specify the size of our compute resources with the `cluster_spec` function. The server 649 has 36 cores and 2 GPUs, which is too many for this demo (by default the cluster tries to use all resources).

I specify 4 CPUs and 0 GPUs. Nothing happens yet, we just created a dictionary of workers that will be created:

In [ ]:
from libertem.executor.dask import cluster_spec, DaskJobExecutor

n_cpus = 4
spec = cluster_spec(cpus=range(n_cpus), cudas=[], has_cupy=False)
print([*spec.keys()])

Now we create the `lt.Context` which is the entry point for most interaction with LiberTEM. To use the cluster specification we also need to tell LiberTEM which backend to use, here `DaskJobExecutor`. If we just called `lt.Context()` it would create a `DaskJobExecutor` cluster with all 36 CPUs and 2 GPUs.

In [ ]:
ctx = lt.Context(DaskJobExecutor.make_local(spec))

The `lt.Context` object has a lot of methods on it, used to load data and run analyses, for example:

- `ctx.load` : loads datasets
- `ctx.run` : runs an `Analysis`
- `ctx.run_udf` : runs a User Defined Function (UDF)
- `ctx.map` : apply a function to every frame in a dataset
- `ctx.create_disk_analysis` : create an `Analysis` object using a circular mask on each frame
- ...

### Load some data

First we load some data.

If the file(s) contain sufficient metadata, then the dataset shape will be automatically inferred, otherwise it might be necessary to set either or both of `nav_shape` and `sig_shape` when calling `load`.

In [ ]:
ds = ctx.load('mib', path=str(dataset_path))

By default LiberTEM supports a range of microscopy data formats, here I have loaded a custom format for the Precession Diffraction datasets used on the PFNC (this is custom code in env_TEM and not currently in the public version of LiberTEM).

The formats supported are (normally):

- Merlin Medipix (MIB)
- Raw binary files
- Digital Micrograph (DM3, DM4) files (image stacks, specifically, DM files are quite varied so not all flavours of DM files are suported)
- EMPAD
- K2IS
- FRMS6
- BLO
- SER
- HDF5
- Norpix SEQ
- MRC
- TVIPS
- Groups of raw files / CEA Precession data format
- Dask arrays

This gives us a dataset object `ds` which represents how the data should be read from disk.

In [ ]:
ds

At this stage no data has been read, the RAM usage for data is 0, we have only defined where the data is stored.

LiberTEM uses 'lazy computation' by default, which means nothing happens until it must happen, e.g. reading a file, and then only what needs to be read or computed is read or computed.

### Nav and Sig dimensions

LiberTEM uses the same concepts as Hyperspy for navigation and signal dimensions.

- Each 'frame' is considered a signal
- Each scan position is considered as a coordinate in the navigation dimensions

**Attention** LiberTEM uses the same conventions as Python/Numpy for array indexing, i.e. `[y, x]` with `(0, 0)` top-left and positive `(down, right)`. Hyperspy reverses this and uses `[x, y]` !!

The dimensions are both available in the `ds.meta` object (which contains many things):

In [ ]:
print(f'Navigation shape: {ds.meta.shape.nav}')

In [ ]:
print(f'Signal shape: {ds.meta.shape.sig}')

One big difference between LiberTEM and HyperSpy datasets is that we cannot (easily) access the data like an array, because of lazy computation and the architecture of the system. This is not possible today, but might be in the future:

In [ ]:
try:
    ds[5, 10]
except TypeError as e:
    print(f'ERROR: {e}')

### Applying a function to a dataset

So far, nothing has happened, but now we will carry out some (simple) computation, summing each frame to get a navigation-shaped result. To do this we need a function to apply:

In [ ]:
def sum_frame(frame):
    return np.sum(frame)

And we run it on the dataset using `ctx.map`:

In [ ]:
pixelsum = ctx.map(dataset=ds, f=sum_frame, progress=True)

With `ctx.map` the result is directly a numpy array which we can plot:

In [ ]:
ax = _plot_img(pixelsum, title='Sum-over-sig')

### Running a UDF

A UDF is a User Defined Function, and is a way of specifying more complex analyses to run on a dataset.

LiberTEM is distributed with a few in-built UDFs:

- SumUDF \[`libertem.udf.sum.SumUDF`\]: Sum of frames every over every navigation coordinate (frame-shaped result)
- LogsumUDF \[`libertem.udf.logsum.LogsumUDF`\]: Logarithmic sum over frames at every navigation coordinate (frame-shaped result)
- SumSigUDF \[`libertem.udf.sumsigudf.SumSigUDF`\]: Get the sum of each frame in a dataset (navigation-shaped result)
- StdDevUDF \[`libertem.udf.stddev.StdDevUDF`\]: Compute the statistics for each pixel over all frames (mean, std, var, sum)
- PickUDF \[`libertem.udf.raw.PickUDF`\]: Load one or more frames from a dataset
- FEMUDF \[`libertem.udf.fem.FEMUDF`\]: Fluctuation EM (standard deviation in ring around zero-order peak)
- HoloReconstructUDF \[`libertem.udf.holography.HoloReconstructUDF`\]: Reconstruct complex electron wave using Fourier transform
- CrystallinityUDF \[`libertem.udf.crystallinity.CrystallinityUDF`\]: Integrate a ring in the Fourier spectrum of each frame

But these mostly exist as examples, and the user should implement what they want to do following the documentation.

Let's run a simple built-in UDF and get the cumulative frame over all scan positions:

In [ ]:
from libertem.udf.sum import SumUDF
udf_results = ctx.run_udf(dataset=ds, udf=SumUDF(), progress=True)

The `ctx.run_udf` method allows us to run one-or-more UDFs on the dataset (minimising the number of times the data is read from disk for each UDF). It can also take the following parameters:

- `roi`: a boolean numpy array which limits the calculation to certain navigation coordinates
- `corrections`: an object specifying things like dark field, gain map etc to pre-process the data before computation

Results from UDFs are not directly a numpy array, as there can be more than one result per UDF, instead a dictionary is returned:

In [ ]:
print([*udf_results.keys()])

Here we have only one result, the intensity of each pixel in the summed frame, the numpy array can be extracted by taking the `.data` attribute of the `'intensity'` result:

In [ ]:
nav_sum_array = udf_results['intensity'].data
print(f'nav_sum_array {type(nav_sum_array)} {nav_sum_array.shape} {nav_sum_array.dtype}')

This syntax is quite verbose but allows LiberTEM to handle many results and very sparse ROIs without making many full-sized copies of the results data.

Now we can plot the result:

In [ ]:
ax = _plot_img(nav_sum_array, figsize=(12, 12), title='Sum-over-nav', cbar=True)

For construction of a UDF from scratch see the final example in this notebook.

### Masked analysis

One of the inbuilt features of LiberTEM is very efficient summation of pixels which lie within a masked region (points, disks, rings). This is motivated by fast virtual detector, BF and ADF imaging.

For the above dataset, we can compute a BF image using the following parameters (all in pixels):

In [ ]:
cx = 123. # centre of zero-order disk (x)
cy = 127. # centre of zero-order disk (y)
radius = 35. # brightfield radius

In [ ]:
ax = _plot_img(nav_sum_array, figsize=(12, 12), title='BF Mask')
disk = plt.Circle((cx, cy), radius, color='r', fill=False, linewidth=2)
ax.add_patch(disk);

With these parameters we create an analysis using the `ctx` and then run it on the dataset:

In [ ]:
bf_analysis = ctx.create_disk_analysis(dataset=ds, cx=cx, cy=cy, r=radius)
bf_results = ctx.run(bf_analysis, progress=True)

The results format is a little different, again we have an `intensity` result but we must access the numpy array using the `raw_data` attribute.

In [ ]:
bf_image = bf_results['intensity'].raw_data

In [ ]:
ax = _plot_img(bf_image, title='Brightfield')

And for completeness here is a virtual ADF image using `ctx.create_ring_analysis` but otherwise the same code:

In [ ]:
adf_analysis = ctx.create_ring_analysis(dataset=ds, cx=cx, cy=cy, ri=radius, ro=radius * 2.)
adf_results = ctx.run(adf_analysis, progress=True)
ax = _plot_img(adf_results['intensity'].raw_data, title='ADF')

### CoM analysis

LiberTEM also has a very fast centre-of-mass implementation, supporting whole-frame, disk and ring CoM with the option to correct for detector / scan rotation in the diffraction patterns.

The same syntax is used, this time through `ctx.create_com_analysis`:

In [ ]:
com_analysis = ctx.create_com_analysis(dataset=ds, cx=cx, cy=cy, mask_radius=radius, scan_rotation=-20.)
com_results = ctx.run(com_analysis, progress=True)

For a CoM result set we have a number of outputs:

In [ ]:
for key in com_results.keys():
    print(f'{key}: {com_results[key].desc}')

Allowing us to compute things like electric field and charge density.

And we can plot the raw data (though this datast is not great for CoM !):

In [ ]:
fig, axs = plt.subplots(1, 2, figsize=(16, 8))
axs[0].imshow(com_results.x.raw_data, cmap='bwr');
axs[0].set_title('X-component');
axs[1].imshow(com_results.y.raw_data, cmap='bwr');
axs[1].set_title('Y-component');

### Diffraction pattern interpretation (LiberTEM-Blobfinder)

An optional add-on for LiberTEM is LiberTEM-Blobfinder, which is specialised in finding diffraction peaks and labelling them via a form of template matching. It can be installed with:

```
pip install libertem-blobfinder
```

On this particular dataset it is not interesting as there is only one peak, but for normal CBED patters this allows us to calculate the lattice vectors and subsequently the strain, or other tasks requiring finding diffraction spots.

For this we need some additional parameters and imports:

In [ ]:
import libertem.utils as ut
import libertem.analysis.gridmatching as grm
import libertem_blobfinder.udf.refinement as rf
from libertem_blobfinder.common import patterns, correlation

true_disk_r = 28.
n_disks = 1
search_radius_multiplier = 1.1
outer_multiplier = 1.1

First we must create our search template, LiberTEM has a few options built in, one of which is a 'background subtraction' mask which works well for this data.

In [ ]:
match_pattern = patterns.BackgroundSubtraction(radius=true_disk_r,
                                               search=true_disk_r * search_radius_multiplier,
                                               radius_outer=true_disk_r * outer_multiplier)

In [ ]:
ax = _plot_img(match_pattern.get_mask(ds.meta.shape.sig), title='Search pattern', figsize=(8, 8))

We now apply this template to one frame to estimate the lattice for the dataset. We use the sum of all frames results computed earlier to increase the SNR:

In [ ]:
peaks_yx = correlation.get_peaks(sum_result=nav_sum_array,
                                 match_pattern=match_pattern,
                                 num_peaks=n_disks)
matcher = grm.Matcher(min_match=1)
match = matcher.affinematch(peaks_yx, indices=np.asarray([[0, 0]]))

The `peaks_yx` object is a numpy array containing the positions `(y, x)` which had a good correlation score with the patterned defined above.

The match object contains information about the matched lattice, specifically the estimated lattice coordinates and a selector object which identifies which peaks in `peaks_yx` are probably part of this lattice.

In [ ]:
coords = match.indices
selector = match.selector

The match object also contains also contains the estimated `zero` order position and `a` and `b` vectors as `(y, x)` arrays.

In [ ]:
zero_yx = match.zero
a_vec = match.a
b_vec = match.b

We can plot these results to see if they correspond to a reasonable lattice:

In [ ]:
ax = _plot_img(nav_sum_array, figsize=(12, 12), title='Lattice')

peaks_yx_plot = peaks_yx[selector, :]
labels = [f'({x}, {y})' for x, y in coords.reshape(-1, 2)]
for (y, x), label in zip(peaks_yx_plot, labels):
    disk = plt.Circle((x, y), true_disk_r, color='r', fill=False, linewidth=2)
    ax.add_patch(disk);
    ax.text(x, y, label, color='r', ha='center', va='center', fontsize='large', fontweight='bold')

a_yx = np.stack([zero_yx, zero_yx + match.a], axis=0)
b_yx = np.stack([zero_yx, zero_yx + match.b], axis=0)
ax.plot(a_yx[:, 1], a_yx[:, 0], color='orange');
ax.plot(b_yx[:, 1], b_yx[:, 0], color='lightgreen');
ax.set_ylim(ds.meta.shape.sig[0], 0);

Now that we have a base lattice, we can fit it to every diffraction pattern in the dataset using the `run_refine` function.

This can take a little while. 

In [ ]:
refine_result, indices_refined = rf.run_refine(ctx=ctx,
                                               dataset=ds,
                                               zero=match.zero,
                                               a=match.a,
                                               b=match.b,
                                               match_pattern=match_pattern,
                                               match='affine',
                                               indices=np.mgrid[0:1, 0:1],
                                               matcher=matcher)

The `indices` argument defines how many steps of the lattice vector the system should search. Here we have one disk so we only provide the (0, 0) index.

`refine_result` has many separate results:

In [ ]:
for key, buf in refine_result.items():
    print(key, buf.data.shape)

The shapes are from the navigation dimension which is `(20, 150)`, with `(7,)` peaks defined and contain an `(y, x)` last dimension of size `(2,)`. 

Within this object we have the refined lattice vectors `a` and `b` for each frame, the zero-order peak position `zero`, as well as evaluations of the fit quality and the peak positions.

- `centers` are the `int` peak positions corresponding to the pixel with maximum correlation score
- `refineds` are `float` coordinates which are the subpixel-refined positions

`selector` is a boolean array indicating which peaks were identified in which diffraction patterns.

Of course, in this dataset with only one diffraction spot the fitted lattice vectors are nonsensical, but otherwise they would have been estimated from the ensemble of detected spots in each frame.

To show something I plot the relative changes to the positions of the central spot X/Y.

In [ ]:
x_pos = refine_result['refineds'].data[..., 0, 1]
x_pos -= refine_result['refineds'].data[..., 0, 1].mean()
ax = _plot_img(x_pos, cmap='bwr', title='Spot X-coord', cbar=True, figsize=(8, 8))

In [ ]:
y_pos = refine_result['refineds'].data[..., 0, 0]
y_pos -= refine_result['refineds'].data[..., 0, 0].mean()
ax = _plot_img(refine_result['refineds'].data[..., 0, 0], cmap='bwr', title='Spot Y-coord', cbar=True, figsize=(8, 8))

### Custom GPU-enabled / CuPy UDF [ADVANCED]

As a (quite complex) example, I'll now demonstrate defining a custom UDF which is also CuPy-enabled (i.e. will use GPU when these are available in the LiberTEM backend), but will function identically (but slower) without a GPU.

This is possible because the API of the library CuPy tries to be as close as possible to that of Numpy, and so in most cases the same code works with both backends. LiberTEM enables this by providing a special variable inside of a UDF: `self.xp` which acts as reference to either CuPy or Numpy as appropriate and available.

The task we will perform with the GPU is trivial (and meaningless), we'll perform a Fourier transform on every frame and return two results:

- the central pixel magnitude for each frame (a navigation-shaped result)
- the sum Fourier transform over all frames (a signal-shaped result)

To make a UDF we must first create a new UDF sub-class. For demonstration purposes I will define the UDF in separate chunks but in real code we would write the complete class definition in one cell.

In [ ]:
from libertem.udf.base import UDF

class FourierMean(UDF):
    pass

We must define some methods before this class will do what we want.

Firstly we must tell LiberTEM that this is a `CuPy`-enabled UDF by defining the compatible backends:

In [ ]:
def get_backends(self):
    return ('numpy', 'cupy',)

FourierMean.get_backends = get_backends

The tuple `('numpy', 'cupy',)` tells LiberTEM that this UDF will run on both numpy and cupy.

Next we must define the results we want to get from running the UDF:

In [ ]:
def get_result_buffers(self):
    return {'central_magnitude': self.buffer(kind='nav', dtype='complex128', where='device'),
            'sum_fourier': self.buffer(kind='sig', dtype='complex128', where='device')}

FourierMean.get_result_buffers = get_result_buffers

A *Buffer* is an array into which we will put our results when running the UDF. The term buffer is used because often we will only be filling / viewing a small part of the whole array, and this corresponds to the behaviour of a block of memory (a buffer) which we allocate before computation and then fill bit-by-bit (and possibly out of order).

The `self.buffer` method allows us to specify the results we will return, and the shape they will have, generally either 'nav' (for navigation / scan sized), or 'sig' (signal or frame size). We also declare the return data type and (in this case) that the result can be computed via a GPU ('device'). We must return these buffers in a dictionary where the key is a string which names the result.

Now we must define what we want to do with each frame, this is the actual code which will be executed to compute our results.

In [ ]:
def process_frame(self, frame):
    # Compute the FFT using self.xp
    frame_fft = self.xp.fft.fft2(frame)
    # Put the results we derive from the FFT into the buffers we declared earlier
    # The [:] ensures that we assign the results into the existing buffer rather than creating a new one
    self.results.central_magnitude[:] = frame_fft[0, 0]
    self.results.sum_fourier[:] += frame_fft        
    
FourierMean.process_frame = process_frame

Here, rather than using `np.fft` directly to perform the fft, we instead use the special property `self.xp` provided by LiberTEM. This property will be either the library `CuPy` when the UDF is being run on a GPU, and will be `numpy` when run on a CPU, allowing the code to work in both contexts. This is possible as CuPy's API is very close to that of Numpy.

We must also tell LiberTEM how to combine results from different parts of the dataset, we do this via the `merge` method. Again note the `[:]` which ensures we are assigning into the buffer rather than re-creating it.

In [ ]:
def merge(self, dest, src):
    dest.central_magnitude[:] = src.central_magnitude
    dest.sum_fourier[:] += src.sum_fourier
    
FourierMean.merge = merge

`dest` and `src` refer to the results for a the whole dataset (destination buffer) and for a single partition (source buffer), respectively.

Finally we can run the UDF. 

NOTE: Our current `lt.Context` only has a few CPU workers so this will act as a baseline for non-GPU execution:

In [ ]:
res = ctx.run_udf(dataset=ds, udf=FourierMean(), progress=True)

To get a comparison, we next create a second context / backend with some GPU workers. Dask/Distributed complains because we already have a cluster running, but this is not a problem.

If running on a machine without CuPy (without a GPU) then the rest of this notebook will fail as we are explicitly creating a CuPy-enabled cluster with no other workers! (normally you would want a mixed cluster).

In [ ]:
spec_gpu = cluster_spec(cpus=[], cudas=[0, 1], has_cupy=True)
ctx_gpu = lt.Context(DaskJobExecutor.make_local(spec_gpu))

In [ ]:
try:
    res = ctx_gpu.run_udf(dataset=ds, udf=FourierMean(), progress=True)
except ModuleNotFoundError:
    print('No CuPy available')

We should see that the GPU-based cluster is much faster for this UDF, even though the code that is running is exactly the same. It will perform optimally on a mixed GPU and CPU cluster with no modifications.

The results are not very interesting, but included for completeness:

In [ ]:
ax = _plot_img(np.fft.fftshift(np.log(np.abs(res['sum_fourier'].data))),
               title='Sum Fourier (log color)',
               figsize=(12, 12))

In [ ]:
ax = _plot_img(np.abs(res['central_magnitude'].data), title='Central Fourier magnitude')